### Setup langfuse tracing

In [ ]:
from my_utils import setup_langfuse_tracer, hf_login
setup_langfuse_tracer()
hf_login()

### My LLM - To act as brain of all the below Agents

In [ ]:
from smolagents import LiteLLMModel
from my_config import MyConfig

my_config = MyConfig()

remote_url = f"http://{my_config.OLLAMA_INFERENCE_NODE_IP}:11434"
model = LiteLLMModel(
    base_url=remote_url
    , model_id='ollama/qwen2.5-coder:7b'
    # , model_id='ollama/llama3.2:latest'
)

### Basic Retrieval with DuckDuckGo

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, FinalAnswerTool

# Initialize the search tool
search_tool = DuckDuckGoSearchTool()
agent = CodeAgent(
    model=model,
    tools=[
        search_tool,
        FinalAnswerTool()
    ],
    model=model,
    max_steps=5,
    verbosity_level=2
)

# Example usage
response = agent.run(
    "Search for luxury superhero-themed party ideas, including decorations, entertainment, and catering."
)
print(response)

### Custom Knowledge Base Tool

#### Simulate Knowledge base

In [ ]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from my_tools import PartyPlanningRetrieverTool
# Simulate a knowledge base about party planning
party_ideas = [
    {"text": "A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.", "source": "Party Ideas 1"},
    {"text": "Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.", "source": "Entertainment Ideas"},
    {"text": "For catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'", "source": "Catering Ideas"},
    {"text": "Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.", "source": "Decoration Ideas"},
    {"text": "Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.", "source": "Entertainment Ideas"}
]

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in party_ideas
]

# Split the documents into smaller chunks for more efficient search
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
docs_processed = text_splitter.split_documents(source_docs)

# Create the retriever tool
party_planning_retriever = PartyPlanningRetrieverTool(docs_processed)

#### Retrieval agent

In [ ]:
from smolagents import CodeAgent, FinalAnswerTool


# Initialize the agent
agent = CodeAgent(
    tools=[
        party_planning_retriever
        , FinalAnswerTool()
    ]
    , model=model
    , max_steps=5
    , verbosity_level=2
)

# Example usage
response = agent.run(
    "Find ideas for a luxury superhero-themed party, including entertainment, catering, and decoration options."
)

print(response)